# Logistic Regression Gradient & Convexity — Student Lab

Complete all TODOs. This lab is math-first and stability-first.

In [1]:
import numpy as np

def check(name: str, cond: bool):
    if not cond:
        raise AssertionError(f'Failed: {name}')
    print(f'OK: {name}')

rng = np.random.default_rng(0)

## Section 0 — Synthetic Dataset
We’ll create a binary classification dataset with both separable and non-separable regimes.

In [2]:
def make_data(n=400, d=5, separable=False):
    X = rng.standard_normal((n, d))
    w_true = rng.standard_normal(d)
    logits = X @ w_true
    if separable:
        y = (logits > 0).astype(int)
    else:
        # add noise so it's not perfectly separable
        logits = logits + 0.5 * rng.standard_normal(n)
        probs = 1 / (1 + np.exp(-logits))
        y = (rng.random(n) < probs).astype(int)
    return X, y, w_true

X, y, w_true = make_data(separable=False)
check('shapes', X.ndim==2 and y.ndim==1 and X.shape[0]==y.shape[0])
X.shape, y.mean()

OK: shapes


((400, 5), np.float64(0.515))

## Section 1 — Sigmoid + Stable Loss

### Task 1.1: Stable sigmoid

# TODO: implement a numerically stable sigmoid.
# HINT:
# - Use `np.where(z>=0, ...)` trick to avoid overflow.

**Explain:** Why does sigmoid saturate for large |z| and what does that do to gradients?

**Answer:** The sigmoid function squeezes any number into a value between 0 and 1. When the input z is very large and positive, sigmoid becomes almost 1. When z is very large and negative, sigmoid becomes almost 0. Beyond a point, increasing or decreasing z hardly changes the output; this is called saturation.

Because the sigmoid curve becomes almost flat in these regions, its slope (gradient) becomes very close to zero. That means the model gets almost no learning signal: changing the weights doesn’t change the output much, so training slows down or can even stall.

In [3]:
def sigmoid(z):
    # TODO
    z = np.asarray(z)
    x = 1 / (1 + np.exp(-z))
    y = np.exp(z) / (1 + np.exp(z))
    return np.where(z >= 0, x, y)

z = np.array([-1000.0, -10.0, 0.0, 10.0, 1000.0])
p = sigmoid(z)
print(p)
check('in_0_1', np.all((p >= 0) & (p <= 1)))
check('monotonic', np.all(np.diff(p) >= 0))

[0.00000000e+00 4.53978687e-05 5.00000000e-01 9.99954602e-01
 1.00000000e+00]
OK: in_0_1
OK: monotonic


/tmp/ipython-input-1819154982.py:4: RuntimeWarning: overflow encountered in exp
  x = 1 / (1 + np.exp(-z))
/tmp/ipython-input-1819154982.py:5: RuntimeWarning: overflow encountered in exp
  y = np.exp(z) / (1 + np.exp(z))
/tmp/ipython-input-1819154982.py:5: RuntimeWarning: invalid value encountered in divide
  y = np.exp(z) / (1 + np.exp(z))


### Task 1.2: Stable binary cross-entropy loss

We use labels y in {0,1}.

Loss per example: `-y log(p) - (1-y) log(1-p)` where p=sigmoid(z).

# TODO: implement stable loss without NaNs/inf.
# HINT:
# - Clip p to [eps, 1-eps]
# - Alternatively use softplus: log(1+exp(z))

**Interview Angle:** explain a stable form of log-loss.

**Answer:** Clipping works, but the “clean” stable approach is to compute BCE from logits z directly, avoiding sigmoid + logs.
For numerical stability, don’t compute BCE using log(sigmoid(z)) directly. Use a logits-based form with softplus (like softplus(z) - y*z), which avoids log(0) and overflow.

In [4]:
def bce_loss(y, p):
    # TODO
    eps = 1e-12
    p = np.clip(p, eps, 1 - eps)
    y = y.astype(float)
    return float(-np.mean(y * np.log(p) + (1 - y)*np.log(1 - p)))

# sanity: loss is finite
z = X @ np.zeros(X.shape[1])
p = sigmoid(z)
L = bce_loss(y, p)
print('loss', L)
check('finite_loss', np.isfinite(L))

loss 0.6931471805599452
OK: finite_loss


## Section 2 — Gradient (Derive → Implement → Check)

### Task 2.1: Derive the gradient (write in markdown)
Show that for loss averaged over n examples:
`grad = X^T (p - y) / n`

**Checkpoint:** What is the shape of grad?

**Answer:** The gradient has the same shape as the weight vector w.

### Task 2.2: Implement loss and gradient

# TODO: implement `logreg_loss_and_grad(X, y, w)` returning (loss, grad).
# HINT:
# - z = X @ w
# - p = sigmoid(z)
# - loss = BCE
# - grad = X.T @ (p - y) / n

**FAANG gotcha:** ensure y is 0/1, not -1/+1.

In [5]:
def logreg_loss_and_grad(X, y, w):
    # TODO
    z = X @ w
    # p = 1 / (1 + np.exp(-z))
    p = sigmoid(z) # using this one since it is more numerically stable
    loss = float(-np.mean(y * np.log(p) + (1 - y) * np.log(1 - p)))
    n = X.shape[0]
    grad = X.T @ (p - y) / n
    return loss, grad

w0 = rng.standard_normal(X.shape[1])
loss, grad = logreg_loss_and_grad(X, y, w0)
print('loss', loss, 'grad_norm', np.linalg.norm(grad))
check('grad_shape', grad.shape == w0.shape)
check('finite', np.isfinite(loss) and np.all(np.isfinite(grad)))

loss 1.3053171072371215 grad_norm 0.5956359579815794
OK: grad_shape
OK: finite


### Task 2.3: Numerical gradient check (finite differences)

# TODO: implement numerical gradient and compare to analytic grad.
# HINT: central difference

**Checkpoint:** Why can eps too small make the check worse?

**Answer:** Because of floating-point precision.

When eps is extremely tiny:
- f(w + epsilon) and f(w - epsilon) become almost identical
- Subtracting them causes you to lose meaningful digits
- The numerator becomes dominated by rounding noise
- Then dividing by 2\epsilon can amplify that noise

So paradoxically: smaller eps does not always mean more accurate.

In short numerical gradient checking works by measuring loss change from tiny weight nudges; if eps is too small, floating-point rounding makes the difference noisy, so the gradient estimate gets worse.

In [6]:
def numerical_grad(f, w, eps=1e-5):
    # TODO
    w = np.asarray(w, dtype=float)
    g = np.zeros_like(w)
    for i in range(w.size):
      e = np.zeros_like(w); e[i] = 1.0
      g[i] = (f(w + eps * e) - f(w - eps * e)) / (2 * eps)
    return g

f = lambda v: logreg_loss_and_grad(X, y, v)[0]
g_num = numerical_grad(f, w0)
_, g = logreg_loss_and_grad(X, y, w0)

max_abs = np.max(np.abs(g - g_num))
print('max_abs_diff', max_abs)
check('grad_check', max_abs < 1e-5)

max_abs_diff 2.187561243260916e-11
OK: grad_check


## Section 3 — Convexity & Hessian Intuition

### Task 3.1: Hessian-vector product (HVP)

For logistic regression:
H = (1/n) X^T S X where S = diag(p(1-p)).

Implement HVP: compute H@v without building full H explicitly.

# HINT:
- s = p*(1-p)
- compute Xv then multiply by s then X^T


In [7]:
def hvp(X, y, w, v):
    # TODO
    z = X @ w
    p = sigmoid(z)
    s = p * (1 - p)
    Xv = X @ v
    return (X.T @ (s * Xv)) / X.shape[0]

v = rng.standard_normal(X.shape[1])
Hv = hvp(X, y, w0, v)
check('hvp_shape', Hv.shape == v.shape)
check('hvp_finite', np.all(np.isfinite(Hv)))

OK: hvp_shape
OK: hvp_finite


### Task 3.2: Empirical PSD check

Check that v^T H v >= 0 for random v (PSD).

**Explain:** Why does PSD imply convexity?

In [8]:
def vHv(X, y, w, v):
    return float(v @ hvp(X, y, w, v))

vals = []
for _ in range(50):
    v = rng.standard_normal(X.shape[1])
    vals.append(vHv(X, y, w0, v))
print('min vHv', min(vals))
check('psd', min(vals) > -1e-8)

min vHv 0.10348656174059763
OK: psd


## Section 4 — Optimization (Bonus)

### Task 4.1: One step of GD vs Newton (conceptual)

Implement one gradient descent step. (Newton step is optional.)

**FAANG gotcha:** perfectly separable data can push weights to infinity; explain why.

**Answer:** Perfectly separable data means that you can draw a line that separates the classes with zero error.

In logistic regression:
- The model can always reduce loss by making predictions more confident
- More confidence = larger weights
- There is no penalty for weights becoming huge (unless regularized)

So the optimizer keeps doing this: Make weights bigger -> predictions closer to 0 or 1 -> loss keeps decreasing

In short gradient descent reduces loss by stepping opposite the gradient, but on perfectly separable data, logistic regression has no finite optimum, so the optimizer keeps pushing weights toward infinity unless regularization is used.

In [10]:
def gd_step(X, y, w, lr):
    # TODO
    _, g = logreg_loss_and_grad(X, y, w)
    return w - lr * g

w1 = gd_step(X, y, w0, lr=0.1)
loss0, _ = logreg_loss_and_grad(X, y, w0)
loss1, _ = logreg_loss_and_grad(X, y, w1)
print('loss0', loss0, 'loss1', loss1)
check('decreased', loss1 <= loss0 + 1e-12)

loss0 1.3053171072371215 loss1 1.2700539510700066
OK: decreased


---
## Submission Checklist
- All TODOs completed
- Gradient check passes
- PSD check passes
- Short answers to checkpoint questions
